In [27]:
import pandas as pd
from tqdm import tqdm
import numpy as np
from gensim.models import Word2Vec
from scipy.spatial import distance
from gluonnlp.data import SentencepieceTokenizer
from kobert.utils import get_tokenizer
from bs4 import BeautifulSoup as bs
from urllib.request import urlopen
from urllib.parse import quote
import requests
import json
import matplotlib.pyplot as plt
%matplotlib inline
import re
import datetime
from datetime import timedelta
from tabulate import tabulate

# Train and fit model to news data

In [28]:
def w2v_train_fit(data, date, standard=0.93):
    theme_vectors_df = pd.read_csv('theme_vectors_xnorm.csv',index_col=0)
    theme_vectors_df['vectors'] = theme_vectors_df['vectors'].apply(lambda x: np.array(eval(x)))
    
    # Set word2vec parameters
    tok_path = get_tokenizer()
    sp  = SentencepieceTokenizer(tok_path)
    v_dimension = 300
    v_window = 8
    model = Word2Vec.load('word2vec.model')
    
    # Get news vectors without normalization
    def vectorize_without_normal(news):
        # Remove letters which are not Hangul
        hangul = re.compile("[^ㄱ-ㅎㅏ-ㅣ가-힣]+")
        news_words = hangul.sub(' ', news)
        # Tokenization with KoBERT tokenizer
        token = sp(news)
        final_tokens = token
        # Vectorization with word2vec
        init_v = np.array([0.0]*v_dimension)
        for word in final_tokens:
            word_vectors = model.wv
            if word in word_vectors.vocab:
                v = model.wv[word]
                init_v = init_v + v
        return init_v
    
    # Apply vectorization and return cosine simmilarity
    def apply_w2v(news):
        news_vec = vectorize_without_normal(news)
        result = []
        # Calculate cosine simmilarity
        for theme in theme_vectors_df['vectors']:
            cosine = 1 - distance.cosine(theme, news_vec)
            result.append(cosine)
        df = pd.DataFrame(data=np.zeros([168,2]), columns=['Theme', 'Result'])
        df['Theme'] = theme_vectors_df['themes']
        df['Result'] = result

        df.sort_values('Result', ascending=False, inplace=True)
        return df

    
    theme_list = []
    cosine_list = []
    for i in tqdm(range(len(data))):
        result = apply_w2v(data['news'][i])
        result = result.sort_values(by='Result', ascending=False).reset_index(drop=True)
        theme = result['Theme'][0]
        cosine = result['Result'][0]
        theme_list.append(theme)
        cosine_list.append(cosine)
        
    df = pd.DataFrame(list(zip(theme_list, cosine_list)), columns = ['Theme', 'Similarity'])
    #df = pd.DataFrame(theme_simil, columns = ['Theme', 'Similarity']) 
    df = pd.concat([data,df],axis=1)
    df_filtered = df[df['Similarity']>=standard].reset_index(drop=True)
    df_filtered = df_filtered[['Theme','news']]
    
    # Filter news whose frequency is more than 0 less than 6
    df_final = df_filtered.groupby('Theme', as_index=False).count().sort_values(by='news',ascending=False)
    df_final = df_final[(df_final['news']>=1) & (df_final['news']<6)].reset_index(drop=True)
    print(date)
    print(tabulate(df_final, headers='keys', tablefmt='psql'))
    
    return df_final

In [29]:
DATA = pd.read_csv("../Data/data_for_use/20200629_news_data.csv",index_col=0)
DATE = '2020-06-29'
result = w2v_train_fit(DATA, DATE)

  0%|          | 0/19943 [00:00<?, ?it/s]

using cached model


100%|██████████| 19943/19943 [03:23<00:00, 97.88it/s] 

2020-06-29
+----+--------------+--------+
|    | Theme        |   news |
|----+--------------+--------|
|  0 | 방역         |      5 |
|  1 | 줄기세포     |      5 |
|  2 | 그린뉴딜     |      2 |
|  3 | Z플립        |      1 |
|  4 | 공공와이파이 |      1 |
|  5 | 돼지열병     |      1 |
|  6 | 엔터테인먼트 |      1 |
|  7 | 키즈         |      1 |
|  8 | 폴더블폰     |      1 |
|  9 | 해저터널     |      1 |
+----+--------------+--------+


# Make Portfolio

In [23]:
def make_portfolio(theme, date):
    # Load Kospi and Kosdaq list
    kospi = pd.read_csv("../Data/stock_data/kospi.csv", encoding='cp949')
    kosdaq = pd.read_csv("../Data/stock_data/kosdaq.csv", encoding='cp949')
    
    # Find target companies which match with theme from theme-comapny list and return compnanies with codes
    def target_com(theme, theme_list):
        company_list = theme_list[theme_list['Theme']==theme]['Company'].iloc[0]
        company_list = str(company_list).replace('[','').replace(']','').replace("\'",'').replace(' ', '').replace('‘','').replace('’','')
        company_list = company_list.split(',')
        
        # Find comapny code of target companies
        target_theme = {}
        for target in company_list:
            try:
                code = kosdaq[kosdaq['기업명']==target].iloc[0]['종목코드']
                code = str(code).zfill(6)
                target_theme[code] = target
            except:
                try:
                    code = kospi[kospi['기업명']==target].iloc[0]['종목코드']
                    code = str(code).zfill(6)
                    target_theme[code] = target
                except: print('Theme list error occurred!')

        return target_theme, company_list

    
    def target_(target_comapny):
        target_theme = {}
        for target in target_company:
            try:
                code = kosdaq[kosdaq['기업명']==target].iloc[0]['종목코드']
                code = str(code).zfill(6)
            except:
                code = kospi[kospi['기업명']==target].iloc[0]['종목코드']
                code = str(code).zfill(6)
            target_theme[code] = target

        return target_theme

    
    # Convert the input date as 5 business days before the date(start_date) and 1 day before the date(end_date)
    def date_convert(date):
        end_date = date

        def date_by_deducting_business_days(from_date):
            business_days_to_deduct = 6
            current_date = from_date
            while business_days_to_deduct > 0:
                current_date += datetime.timedelta(days=-1)
                weekday = current_date.weekday()
                if weekday >= 5: # sunday = 6
                    continue
                business_days_to_deduct -= 1
            return current_date

        convert_end_date = datetime.datetime.strptime(date,'%Y-%m-%d').date()
        start_date=datetime.date.strftime(date_by_deducting_business_days(convert_end_date),'%Y-%m-%d')

        return start_date, end_date

    # Date format
    def date_format(d):
        d = str(d).replace('-', '.')
        yyyy = int(d.split('.')[0])
        mm = int(d.split('.')[1])
        dd = int(d.split('.')[2])

        this_date = dt.date(yyyy, mm, dd)
        return this_date
    def date_format(d=''):
        if d != '':
            this_date = pd.to_datetime(d).date()
        else:
            this_date = pd.Timestamp.today().date()
        return (this_date)


    # Get the information of stock from NAVER
    def stock_info(stock_cd):
        url_float = 'http://companyinfo.stock.naver.com/v1/company/c1010001.aspx?cmp_cd=' + stock_cd
        source = urlopen(url_float).read()
        soup = bs(source, 'lxml')

        tmp = soup.find(id='cTB11').find_all('tr')[6].td.text
        tmp = tmp.replace('\r', '').replace('\n','').replace('\t','')
        tmp = re.split('/', tmp)

        outstanding = tmp[0].replace(',','').replace('주','').replace(' ','')
        outstanding = int(outstanding)

        floating = tmp[1].replace(' ','').replace('%','')
        floating = float(floating)

        name = soup.find(id='pArea').find('div').find('div').find('tr').find('td').find('span').text

        stock_outstanding[stock_cd] = outstanding
        stock_floating[stock_cd] = floating
        stock_name[stock_cd] = name

        return oustanding, floating, name


    # Get the historical price of index from NAVER
    def historical_index_naver(index_cd, start_date='', end_date='', page_n=1, last_page=0):
        if start_date:
            start_date = date_format(start_date)
        else:
            start_date = datetime.date.today()
        if end_date:
            end_date = date_format(end_date)+datetime.timedelta(days=-1)
        else:
            end_date = datetime.date.today()

        naver_index = 'http://finance.naver.com/sise/sise_index_day.nhn?code=' + index_cd + '&page=' + str(page_n)

        source = urlopen(naver_index).read()
        source = bs(source, 'lxml')

        dates = source.find_all('td', class_='date')
        prices = source.find_all('td', class_='number_1')
        #historical_prices = dict()


        for n in range(len(dates)):
            if dates[n].text.split('.')[0].isdigit():

                # Handling date
                this_date = dates[n].text
                this_date = date_format(this_date)

                if this_date <= end_date and this_date >= start_date:

                    # Handling closing price
                    this_close = prices[n*4].text
                    this_close = this_close.replace(',','')
                    this_close = float(this_close)

                    historical_prices[this_date] = this_close

                elif this_date < start_date:
                    return historical_prices

        # Navigating page
        if last_page == 0:
            last_page = source.find('td', class_='pgRR').find('a')['href']

            last_page = last_page.split('&')[1]
            last_page = last_page.split('=')[1]
            last_page = int(last_page)

        if page_n < last_page:
            page_n += 1
            historical_index_naver(index_cd, start_date, end_date, page_n, last_page)

        return historical_prices

    
    # Get the historical price of stock from NAVER
    def historical_stock_naver(stock_cd, start_date='', end_date='', page_n=1, last_page=0):
        if start_date:
            start_date = date_format(start_date)
        else:
            start_date = datetime.date.today()
        if end_date:
            end_date = date_format(end_date)+datetime.timedelta(days=-1)
        else:
            end_date = datetime.date.today()

        naver_stock = 'http://finance.naver.com/item/sise_day.nhn?code=' + stock_cd + '&page=' + str(page_n)

        source = urlopen(naver_stock).read()
        #source = requests.get(naver_stock)
        source = bs(source, 'lxml')

        dates = source.find_all('span', class_='tah p10 gray03')
        prices = source.find_all('td', class_='num')
        #historical_prices = dict()

        for n in range(len(dates)):

            if len(dates) > 0:

                # Handling date
                this_date = dates[n].text
                this_date = date_format(this_date)

                if this_date <= end_date and this_date >= start_date:

                    # Handling closing price
                    this_close = prices[n*6].text
                    this_close = this_close.replace(',','')
                    this_close = float(this_close)

                    historical_prices[this_date] = this_close

                elif this_date < start_date:
                    return historical_prices


        # Navigating page
        if last_page == 0:
            last_page = source.find('td', class_='pgRR').find('a')['href']
            last_page = last_page.split('&')[1]
            last_page = last_page.split('=')[1]
            last_page = int(last_page)

        if page_n < last_page:
            page_n += 1
            historical_stock_naver(stock_cd, start_date, end_date, page_n, last_page)

        return historical_prices


    # Set threshold of variance to select stable stock
    def apply_threshold(var, threshold=0.05):
        if var < -1*threshold or var > threshold: return False
        else : return True


    def stock_get_info(target_theme):
        stock_outstanding = dict()
        stock_floating = dict()
        stock_name = dict()
        for stock_cd in target_theme.keys():
            stock_info(stock_cd)

        return stock_outstanding, stock_floating, stock_name

    
    start_date, end_date = date_convert(date)
    theme_list=pd.read_csv("../Data/data_for_use/theme_company.csv",index_col=0)
    target_theme, company_list = target_com(theme, theme_list)

    theme_historical_prices = dict()

    for stock_cd in target_theme.keys():
        historical_prices = dict()
        historical_stock_naver(stock_cd, start_date, end_date)
        theme_historical_prices[stock_cd] = historical_prices

    theme_historical_price = pd.DataFrame(theme_historical_prices)
    theme_historical_price.sort_index(axis=1, inplace=True)

    theme_historical_price = theme_historical_price.fillna(method='ffill')
    if theme_historical_price.isnull().values.any():
        theme_historical_price = theme_historical_price.fillna(method='bfill')

    theme_historical_price['kospi200'] = historical_index_naver('KPI200', start_date, end_date).values()
    
    df = theme_historical_price
    # Check change rate
    for i in range(len(df.index)-1):
        df.iloc[i] = (theme_historical_price.iloc[i+1] - theme_historical_price.iloc[i])/theme_historical_price.iloc[i]
    
    df.drop(index=[df.index[-1]], inplace=True)


    print('\n')
    print(f'{date}의 추천테마는 : {theme}')
    print('해당 테마의 추천 종목은 :')
    result = {}
    # Check whether comapny met fluctuation threshold
    for i in range(len(company_list)):
        standard = 1
        try:
            for j in range(len(df.index)-1):
                standard *= apply_threshold(df.iloc[:, i][j])

            if standard == 1 : 
                print('{} : {}'.format(df.columns[i], target_theme[df.columns[i]]))
                result[df.columns[i]] = target_theme[df.columns[i]]
        except:
            print('Fluctuation check failed')
            pass
        
    return result

In [24]:
portfolio = make_portfolio('해저터널', DATE)



2020-06-29의 추천테마는 : 해저터널
해당 테마의 추천 종목은 :
028100 : 동아지질
060370 : KT서브마린


# Backtest

In [109]:
def backtest(portfolio, date):
    
    def date_convert(date):
        start_date = date

        def date_by_adding_business_days(from_date):
            business_days_to_add = 6
            current_date = from_date
            while business_days_to_add > 0:
                current_date += datetime.timedelta(days=1)
                weekday = current_date.weekday()
                if weekday >= 5: # sunday = 6
                    continue
                business_days_to_add -= 1
            return current_date

        convert_start_date = datetime.datetime.strptime(date,'%Y-%m-%d').date()
        end_date=datetime.date.strftime(date_by_adding_business_days(convert_start_date),'%Y-%m-%d')

        return start_date, end_date
    
    
    def date_format(d):
        d = str(d).replace('-', '.')
        yyyy = int(d.split('.')[0])
        mm = int(d.split('.')[1])
        dd = int(d.split('.')[2])

        this_date = dt.date(yyyy, mm, dd)
        return this_date
    def date_format(d=''):
        if d != '':
            this_date = pd.to_datetime(d).date()
        else:
            this_date = pd.Timestamp.today().date()
        return (this_date)

    def historical_stock_naver(stock_cd, start_date='', end_date='', page_n=1, last_page=0):
    
        if start_date:
            start_date = date_format(start_date)
        else:
            start_date = datetime.date.today()
        if end_date:
            end_date = date_format(end_date)+datetime.timedelta(days=-1)
        else:
            end_date = datetime.date.today()

        naver_stock = 'http://finance.naver.com/item/sise_day.nhn?code=' + stock_cd + '&page=' + str(page_n)

        source = urlopen(naver_stock).read()
        #source = requests.get(naver_stock)
        source = bs(source, 'lxml')

        dates = source.find_all('span', class_='tah p10 gray03')
        prices = source.find_all('td', class_='num')
        #historical_prices = dict()

        for n in range(len(dates)):

            if len(dates) > 0:

                # Handle date
                this_date = dates[n].text
                this_date = date_format(this_date)

                if this_date <= end_date and this_date >= start_date:

                    # Handle closing price
                    this_close = prices[n*6].text
                    this_close = this_close.replace(',','')
                    this_close = float(this_close)

                    historical_prices[this_date] = this_close

                elif this_date < start_date:
                    return historical_prices

        # Navigate page
        if last_page == 0:
            last_page = source.find('td', class_='pgRR').find('a')['href']
            last_page = last_page.split('&')[1]
            last_page = last_page.split('=')[1]
            last_page = int(last_page)

        if page_n < last_page:
            page_n += 1
            historical_stock_naver(stock_cd, start_date, end_date, page_n, last_page)

        return historical_prices

    start_date, end_date = date_convert(date)

    theme_historical_prices = dict()
    
    for stock_cd in portfolio.keys():
        historical_prices = dict()
        historical_stock_naver(stock_cd, start_date, end_date)
        theme_historical_prices[stock_cd] = historical_prices

    theme_historical_price = pd.DataFrame(theme_historical_prices)
    theme_historical_price.sort_index(axis=1, inplace=True)

    theme_historical_price = theme_historical_price.fillna(method='ffill')
    if theme_historical_price.isnull().values.any():
        theme_historical_price = theme_historical_price.fillna(method='bfill')


    #theme_historical_price['kospi200'] = historical_index_naver('KPI200', start_date, end_date).values()
    
    df = theme_historical_price
    # Check increase/decrease rate
    for i in range(len(df.index)-1):
        df.iloc[i] = (theme_historical_price.iloc[i] - theme_historical_price.iloc[-1])/theme_historical_price.iloc[-1]
    df.drop(index=[df.index[-1]], inplace=True)
    
    # Apply selling policy: sell if 5% decrease or 10% increase
    for i in range(len(df.columns)):
        for j in range(1, len(df.index)+1):
            if (df.iloc[-j, i]>=0.1) or (df.iloc[-j, i]<=-0.05):
                df.iloc[0:-j, i] = df.iloc[-j, i]
    print(tabulate(df, headers='keys', tablefmt='psql'))
    print(f'Average IRR: {round(df.iloc[0].mean()*100,2)}% for {len(df.index)} days')
    return df.iloc[0].mean()

In [110]:
backtest(portfolio, DATE)

+------------+-----------+-------------+
|            |    028100 |      060370 |
|------------+-----------+-------------|
| 2020-07-03 | 0.0611511 |  0.0521845  |
| 2020-07-02 | 0.0215827 |  0.0424757  |
| 2020-07-01 | 0.0143885 |  0.0291262  |
| 2020-06-30 | 0.0107914 | -0.00364078 |
+------------+-----------+-------------+
Average IRR: 5.67% for 4 days


0.05666777257805406

In [113]:
DATA = pd.read_csv("../Data/data_for_use/20200629_news_data.csv",index_col=0)
DATE = '2020-06-29'
w2v = w2v_train_fit(DATA, DATE)
portfolio = make_portfolio(w2v.iloc[-2,0], DATE)
result = backtest(portfolio, DATE)

  0%|          | 0/19943 [00:00<?, ?it/s]

using cached model


100%|██████████| 19943/19943 [03:45<00:00, 88.41it/s] 


2020-06-29
+----+--------------+--------+
|    | Theme        |   news |
|----+--------------+--------|
|  0 | 방역         |      5 |
|  1 | 줄기세포     |      5 |
|  2 | 그린뉴딜     |      2 |
|  3 | Z플립        |      1 |
|  4 | 공공와이파이 |      1 |
|  5 | 돼지열병     |      1 |
|  6 | 엔터테인먼트 |      1 |
|  7 | 키즈         |      1 |
|  8 | 폴더블폰     |      1 |
|  9 | 해저터널     |      1 |
+----+--------------+--------+


2020-06-29의 추천테마는 : 폴더블폰
해당 테마의 추천 종목은 :
055490 : 테이팩스
060720 : KH바텍
131760 : 파인텍
148150 : 세경하이테크
179900 : 유티아이
+------------+-------------+-------------+------------+------------+-------------+
|            |      055490 |      060720 |     131760 |     148150 |      179900 |
|------------+-------------+-------------+------------+------------+-------------|
| 2020-07-03 |  0.0215517  |  0.00653595 | 0.0501567  |  0.0418327 | -0.0369231  |
| 2020-07-02 | -0.00862069 |  0.0130719  | 0.00626959 |  0.0418327 | -0.04       |
| 2020-07-01 | -0.00646552 |  0.00653595 | 0          |  0.03187